In [3]:
import neurokit2 as nk
import pandas as pd
from scipy.stats import skew, kurtosis
import numpy as np

In [4]:
import pickle
dict_emotion = pickle.load(open('dict_emotion_file', 'rb'))

In [7]:
sampling_rate=1000

In [11]:
signal=dict_emotion['Happy'][0]['C026'][:,0]

In [12]:
clean = nk.emg_clean(signal,sampling_rate=sampling_rate)
amplitude = nk.emg_amplitude(clean)

In [13]:
len(clean)

607963

In [42]:
activity, info_act = nk.emg_activation(emg_cleaned=clean[0:606905],sampling_rate=sampling_rate,method='biosppy')

### Pre-processing

In [9]:
def signal_process (signal, label, sampling_rate, a):
    
    if label == 'EMG':
        clean = nk.emg_clean(signal,sampling_rate=sampling_rate)
        amplitude = nk.emg_amplitude(clean)
        activity, info_act = nk.emg_activation(emg_cleaned=clean[0:a],sampling_rate=sampling_rate,method='biosppy')
        
        # Duration of Activity, Max peak of activity, Mean peaks of activity
        duration_activity=[]
        peak_activity=[]
        mean_activity=[]
        area_activity=[]
        amplitude_activity=[]
        for i in range(0,len(info_act['EMG_Offsets'])):
            duration=(info_act['EMG_Offsets'][i]-info_act['EMG_Onsets'][i])/sampling_rate # in seconds
            peak=max(amplitude[info_act['EMG_Onsets'][i]:info_act['EMG_Offsets'][i]])
            mean=np.mean(amplitude[info_act['EMG_Onsets'][i]:info_act['EMG_Offsets'][i]])
            area=np.trapz(amplitude[info_act['EMG_Onsets'][i]:info_act['EMG_Offsets'][i]])            
            duration_activity.append(duration) #The duration of the muscle activity
            peak_activity.append(peak) #The maximum peak of the muscle activity
            mean_activity.append(mean) #The mean of peaks of the muscle activity
            area_activity.append(area) #The area of muscle activity
            amplitude_activity.append(amplitude[info_act['EMG_Onsets'][i]:info_act['EMG_Offsets'][i]]) # redundante
        
       
        amplitude_activity = [item for sublist in amplitude_activity for item in sublist]
    
        return duration_activity, peak_activity, mean_activity, area_activity, amplitude_activity
                    
    elif label == 'EDA':
        eda_feat, info_eda=nk.eda_process(eda_signal=signal, sampling_rate=sampling_rate)
        eda_symp = nk.eda_sympathetic(eda_feat['EDA_Raw'],sampling_rate=sampling_rate,frequency_band=[0.045,0.25],method='posada')
        #raw_signal=eda_feat['EDA_Raw']
        eda_tonic = eda_feat['EDA_Tonic'] # The tonic component of eda
        eda_phasic = eda_feat['EDA_Phasic'] # The phasic component of eda
        scr_height=info_eda['SCR_Height'] #The SCR amplitude including tonic component
        scr_amplitude=info_eda['SCR_Amplitude'] #The SCR amplitude excluding tonic component
        scr_risetime=info_eda['SCR_RiseTime'] #The time is taken for SCR onset to reach peak amplitude
        scr_recoverytime=info_eda['SCR_RecoveryTime'] #The time it takes for SCR to decrease to half amplitude.
        
        return eda_symp, scr_height, scr_amplitude, scr_risetime, scr_recoverytime, eda_tonic, eda_phasic
        
    elif label == 'ECG':
        ecg_clean = nk.ecg_clean(signal,sampling_rate=sampling_rate)
        ecg_Rpeaks, info_Rpeaks = nk.ecg_peaks(ecg_clean,sampling_rate=sampling_rate) #signal to ecg_clean
        #peaks_cwt=nk.ecg_delineate(ecg_clean, info_peaks['ECG_R_Peaks'], sampling_rate, method='cwt')
        #peaks_dwt=nk.ecg_delineate(ecg_clean, info_peaks['ECG_R_Peaks'], sampling_rate, method='dwt')
        ecg_peaks, info_peaks = nk.ecg_delineate(ecg_clean, info_Rpeaks['ECG_R_Peaks'], sampling_rate)
        ecg_rate = nk.ecg_rate(ecg_Rpeaks, sampling_rate=sampling_rate)
        
        t_duration=[]
        for i in range(0,len(info_peaks['ECG_T_Offsets'])):
            duration=(info_peaks['ECG_T_Offsets'][i]-info_peaks['ECG_T_Peaks'][i])/1000 # in seconds
            t_duration.append(duration) #The duration between the T peak and the offset
           
        return ecg_Rpeaks, ecg_rate, t_duration
    

### Feature Extraction

In [47]:
def feature_extraction (dic, label, sampling_rate):
    
    if label == 'EMG_MF':
        mean_dur, standard_deviation_dur, variance_dur, skewness_dur, kurtis_dur = statistics (dic['MF_duration_activity'])
        mean_peak, standard_deviation_peak, variance_peak, skewness_peak, kurtis_peak = statistics (dic['MF_peak_activity'])
        mean_act, standard_deviation_act, variance_act, skewness_act, kurtis_act = statistics (dic['MF_mean_activity'])
        mean_amp, standard_deviation_amp, variance_amp, skewness_amp, kurtis_amp = statistics (dic['MF_amplitude_activity'])
        mean_area, standard_deviation_area, variance_area, skewness_area, kurtis_area = statistics (dic['MF_area_activity'])
        
        EMG_Activations_N=len(dic['MF_duration_activity'])
                
        features_emg=pd.DataFrame({'EMG_MF_Activations_N':[EMG_Activations_N],
                                   'EMG_MF_Duration_Mean':[mean_dur],'EMG_MF_Duration_Std':[standard_deviation_dur],
                                   'EMG_MF_Duration_Var':[variance_dur],'EMG_MF_Duration_Skew':[skewness_dur],
                                   'EMG_MF_Duration_Kurt':[kurtis_dur],
                                   'EMG_MF_MaxPeakAct_Mean':[mean_peak],'EMG_MF_MaxPeakAct_Std':[standard_deviation_peak],
                                   'EMG_MF_MaxPeakAct_Var':[variance_peak],'EMG_MF_MaxPeakAct_Skew':[skewness_peak],
                                   'EMG_MF_MaxPeakAct_Kurt':[kurtis_peak],
                                   'EMG_MF_MeanPeaksAct_Mean':[mean_act],'EMG_MF_MeanPeaksAct_Std':[standard_deviation_act],
                                   'EMG_MF_MeanPeaksAct_Var':[variance_act],'EMG_MF_MeanPeaksAct_Skew':[skewness_act],
                                   'EMG_MF_MeanPeaksAct_Kurt':[kurtis_act],
                                   'EMG_MF_all_Amplitude_Mean':[mean_amp],'EMG_MF_all_Amplitude_Std':[standard_deviation_amp],
                                   'EMG_MF_all_Amplitude_Var':[variance_amp],'EMG_MF_all_Amplitude_Skew':[skewness_amp],
                                   'EMG_MF_all_Amplitude_Kurt':[kurtis_amp],
                                   'EMG_MF_Area_Mean':[mean_amp],'EMG_MF_Area_Std':[standard_deviation_amp],
                                   'EMG_MF_Area_Var':[variance_amp],'EMG_MF_Area_Skew':[skewness_amp],
                                   'EMG_MF_Area_Kurt':[kurtis_amp]})
        
        
        return features_emg
    
    elif label == 'EMG_TR':
        mean_dur, standard_deviation_dur, variance_dur, skewness_dur, kurtis_dur = statistics (dic['TR_duration_activity'])
        mean_peak, standard_deviation_peak, variance_peak, skewness_peak, kurtis_peak = statistics (dic['TR_peak_activity'])
        mean_act, standard_deviation_act, variance_act, skewness_act, kurtis_act = statistics (dic['TR_mean_activity'])
        mean_amp, standard_deviation_amp, variance_amp, skewness_amp, kurtis_amp = statistics (dic['TR_amplitude_activity'])
        mean_area, standard_deviation_area, variance_area, skewness_area, kurtis_area = statistics (dic['TR_area_activity'])
        
        EMG_Activations_N=len(dic['TR_duration_activity'])
                
        features_emg=pd.DataFrame({'EMG_TR_Activations_N':[EMG_Activations_N],
                                   'EMG_TR_Duration_Mean':[mean_dur],'EMG_TR_Duration_Std':[standard_deviation_dur],
                                   'EMG_TR_Duration_Var':[variance_dur],'EMG_TR_Duration_Skew':[skewness_dur],
                                   'EMG_TR_Duration_Kurt':[kurtis_dur],
                                   'EMG_TR_MaxPeakAct_Mean':[mean_peak],'EMG_TR_MaxPeakAct_Std':[standard_deviation_peak],
                                   'EMG_TR_MaxPeakAct_Var':[variance_peak],'EMG_TR_MaxPeakAct_Skew':[skewness_peak],
                                   'EMG_TR_MaxPeakAct_Kurt':[kurtis_peak],
                                   'EMG_TR_MeanPeaksAct_Mean':[mean_act],'EMG_TR_MeanPeaksAct_Std':[standard_deviation_act],
                                   'EMG_TR_MeanPeaksAct_Var':[variance_act],'EMG_TR_MeanPeaksAct_Skew':[skewness_act],
                                   'EMG_TR_MeanPeaksAct_Kurt':[kurtis_act],
                                   'EMG_TR_all_Amplitude_Mean':[mean_amp],'EMG_TR_all_Amplitude_Std':[standard_deviation_amp],
                                   'EMG_TR_all_Amplitude_Var':[variance_amp],'EMG_TR_all_Amplitude_Skew':[skewness_amp],
                                   'EMG_TR_all_Amplitude_Kurt':[kurtis_amp],
                                   'EMG_TR_Area_Mean':[mean_amp],'EMG_TR_Area_Std':[standard_deviation_amp],
                                   'EMG_TR_Area_Var':[variance_amp],'EMG_TR_Area_Skew':[skewness_amp],
                                   'EMG_TR_Area_Kurt':[kurtis_amp]})
        
        return features_emg
    
    elif label=='EDA':
        symp=pd.DataFrame({'EDA_Symp':[dic['eda_symp']['EDA_Symp']],'EDA_SympN':[dic['eda_symp']['EDA_SympN']]})
        
        mean_ton, standard_deviation_ton, variance_ton, skewness_ton, kurtis_ton = statistics (dic['eda_tonic'])
        mean_pha, standard_deviation_pha, variance_pha, skewness_pha, kurtis_pha = statistics (dic['eda_phasic'])
        
        mean_hei, standard_deviation_hei, variance_hei, skewness_hei, kurtis_hei = statistics (dic['scr_height'])
        mean_amp, standard_deviation_amp, variance_amp, skewness_amp, kurtis_amp = statistics (dic['scr_amplitude'])
        mean_rise, standard_deviation_rise, variance_rise, skewness_rise, kurtis_rise = statistics (dic['scr_risetime'])
        mean_rec, standard_deviation_rec, variance_rec, skewness_rec, kurtis_rec = statistics (dic['scr_recoverytime'])
        
        SCR_Peaks_N=len(dic['scr_height'])
        
        statistics_eda=pd.DataFrame({'SCR_Peaks_N':[SCR_Peaks_N],
                                     'EDA_Tonic_Mean':[mean_ton],'EDA_Tonic_Std':[standard_deviation_ton],
                                     'EDA_Tonic_Var':[variance_ton],'EDA_Tonic_Skew':[skewness_ton],
                                     'EDA_Tonic_Kurt':[kurtis_ton],
                                     'EDA_Phasic_Mean':[mean_pha],'EDA_Phasic_Std':[standard_deviation_pha],
                                     'EDA_Phasic_Var':[variance_pha],'EDA_Phasic_Skew':[skewness_pha],
                                     'EDA_Phasic_Kurt':[kurtis_pha],
                                     'SCR_Height_Mean':[mean_hei],'SCR_Height_Std':[standard_deviation_hei],
                                     'SCR_Height_Var':[variance_hei],'SCR_Height_Skew':[skewness_hei],
                                     'SCR_Height_Kurt':[kurtis_hei],
                                     'SCR_Amplitude_Mean':[mean_amp],'SCR_Amplitude_Std':[standard_deviation_amp],
                                     'SCR_Amplitude_Var':[variance_amp],'SCR_Amplitude_Skew':[skewness_amp],
                                     'SCR_Amplitude_Kurt':[kurtis_amp],
                                     'SCR_RiseTime_Mean':[mean_rise],'SCR_RiseTime_Std':[standard_deviation_rise],
                                     'SCR_RiseTime_Var':[variance_rise],'SCR_RiseTime_Skew':[skewness_rise],
                                     'SCR_RiseTime_Kurt':[kurtis_rise],
                                     'SCR_RecoveryTime_Mean':[mean_rec],'SCR_RecoveryTime_Std':[standard_deviation_rec],
                                     'SCR_RecoveryTime_Var':[variance_rec],'SCR_RecoveryTime_Skew':[skewness_rec],
                                     'SCR_RecoveryTime_Kurt':[kurtis_rec]})
        
        features_eda=pd.concat([symp, statistics_eda],axis=1)
        
        return features_eda
    
    elif label == 'ECG':      
        hrv_feat = nk.hrv(dic['ecg_Rpeaks'], sampling_rate=sampling_rate)
        
        mean_rate, standard_deviation_rate, variance_rate, skewness_rate, kurtis_rate = statistics (dic['ecg_rate'])
        mean_dur, standard_deviation_dur, variance_dur, skewness_dur, kurtis_dur = statistics (dic['t_duration'])
        
        statistics_ecg=pd.DataFrame({'ECG_Rate_Mean':[mean_rate],'ECG_Rate_Std':[standard_deviation_rate],
                                     'ECG_Rate_Var':[variance_rate],'ECG_Rate_Skew':[skewness_rate],
                                     'ECG_Rate_Kurt':[kurtis_rate],
                                     'ECG_Tduration_Mean':[mean_dur],'ECG_Tduration_Std':[standard_deviation_dur],
                                     'ECG_Tduration_Var':[variance_dur],'ECG_Tduration_Skew':[skewness_dur],
                                     'ECG_Tduration_Kurt':[kurtis_dur],})
        
        features_ecg=pd.concat([statistics_ecg, hrv_feat],axis=1)
           
        return features_ecg
    

### Statistics

In [18]:
def statistics (a):
    mean=np.nanmean(a)
    standard_deviation=np.nanstd(a)
    variance=np.nanvar(a)
    skewness = (3*(mean-np.nanmedian(a)))/standard_deviation
    #skewness=skew(a, nan_policy='omit')
    kurtis=kurtosis(a, nan_policy='omit')
    
    return mean, standard_deviation, variance, skewness, kurtis